In [1]:
import os
HOME = os.getcwd()
print(HOME)

/root


In [2]:
if not os.path.exists("TrackNetV3"):
    !git clone https://github.com/lu-joseph/TrackNetV3
%cd {HOME}/TrackNetV3
if not os.path.exists("ckpts"):
    !wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA&export=download&authuser=0&confirm=t&uuid=fd5c99d1-dc6a-4011-948b-1d03be7d6628&at=AN_67v1rreyiyg-hMrVlkh9Np5is%3A1729497024731' -O TrackNetV3_ckpts.zip
    !unzip TrackNetV3_ckpts.zip
%cd {HOME}

/root/TrackNetV3
/root


In [3]:
%cd {HOME}/TrackNetV3
!pip install -r requirements.txt
%cd {HOME}

/root/TrackNetV3


  Using cached dash-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached opencv_python-4.4.0.46-cp38-cp38-manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached pandas-2.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached Pillow-10.0.0-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached plotly-5.8.2-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached parse-1.20.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached Flask_Compress-1.15-py3-none-any.whl.metadata (8.4 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached werkzeug-3.0.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.wh

In [4]:
import math
HEIGHT = 288
WIDTH = 512
SIGMA = 2.5
DELTA_T = 1/math.sqrt(HEIGHT**2 + WIDTH**2)
COOR_TH = DELTA_T * 50
IMG_FORMAT = 'png'

In [5]:
%cd {HOME}/TrackNetV3
from predict import predict
from utils.general import get_model
import torch
import cv2

video_file = '../rally.mp4'
video_name = video_file.split('/')[-1][:-4]
tracknet_file = 'ckpts/TrackNet_best.pt'
inpaintnet_file = 'ckpts/InpaintNet_best.pt' 
save_dir = "prediction"
if not os.path.exists(save_dir):
        os.makedirs(save_dir)
out_csv_file = os.path.join(save_dir, f'{video_name}_ball.csv')
batch_size = 12
num_workers = batch_size if batch_size <= 16 else 16
eval_mode = "weight"


tracknet_ckpt = torch.load(tracknet_file)
tracknet_seq_len = tracknet_ckpt['param_dict']['seq_len']
bg_mode = tracknet_ckpt['param_dict']['bg_mode']
tracknet = get_model('TrackNet', tracknet_seq_len, bg_mode).cuda()
tracknet.load_state_dict(tracknet_ckpt['model'])

inpaintnet_ckpt = torch.load(inpaintnet_file)
inpaintnet_seq_len = inpaintnet_ckpt['param_dict']['seq_len']
inpaintnet = get_model('InpaintNet').cuda()
inpaintnet.load_state_dict(inpaintnet_ckpt['model'])

cap = cv2.VideoCapture(video_file)
w, h = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
w_scaler, h_scaler = w / WIDTH, h / HEIGHT
img_scaler = (w_scaler, h_scaler)

tracknet_pred_dict = {'Frame':[], 'X':[], 'Y':[], 'Visibility':[], 'Inpaint_Mask':[],
                    'Img_scaler': (w_scaler, h_scaler), 'Img_shape': (w, h)}

/root/TrackNetV3


/tmp/ipykernel_34/2260988115.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tracknet_ckpt = torch.load(tracknet_file)
/tmp/ipykernel_34/2260988115.py:26: FutureWarning

In [6]:
%cd {HOME}/TrackNetV3
from utils.general import generate_frames
from dataset import Shuttlecock_Trajectory_Dataset, Video_IterableDataset
from torch.utils.data import DataLoader
import numpy as np
tracknet.eval()
seq_len = tracknet_seq_len

# Sample all frames from video
frame_list = generate_frames(video_file)
dataset = Shuttlecock_Trajectory_Dataset(seq_len=seq_len, sliding_step=1, data_mode='heatmap', bg_mode=bg_mode,
                                        frame_arr=np.array(frame_list)[:, :, :, ::-1])
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False)
video_len = len(frame_list)

/root/TrackNetV3


In [7]:
%cd {HOME}/TrackNetV3
from utils.test import get_ensemble_weight
from tqdm import tqdm
# Init prediction buffer params
num_sample, sample_count = video_len-seq_len+1, 0
buffer_size = seq_len - 1
batch_i = torch.arange(seq_len) # [0, 1, 2, 3, 4, 5, 6, 7]
frame_i = torch.arange(seq_len-1, -1, -1) # [7, 6, 5, 4, 3, 2, 1, 0]
y_pred_buffer = torch.zeros((buffer_size, seq_len, HEIGHT, WIDTH), dtype=torch.float32)
weight = get_ensemble_weight(seq_len, eval_mode)
for step, (i, x) in enumerate(tqdm(data_loader)):
    x = x.float().cuda()
    b_size, seq_len = i.shape[0], i.shape[1]
    with torch.no_grad():
        y_pred = tracknet(x).detach().cpu()
    
    y_pred_buffer = torch.cat((y_pred_buffer, y_pred), dim=0)
    ensemble_i = torch.empty((0, 1, 2), dtype=torch.float32)
    ensemble_y_pred = torch.empty((0, 1, HEIGHT, WIDTH), dtype=torch.float32)

    for b in range(b_size):
        if sample_count < buffer_size:
            # Imcomplete buffer
            y_pred = y_pred_buffer[batch_i+b, frame_i].sum(0) / (sample_count+1)
        else:
            # General case
            y_pred = (y_pred_buffer[batch_i+b, frame_i] * weight[:, None, None]).sum(0)
        
        ensemble_i = torch.cat((ensemble_i, i[b][0].reshape(1, 1, 2)), dim=0)
        ensemble_y_pred = torch.cat((ensemble_y_pred, y_pred.reshape(1, 1, HEIGHT, WIDTH)), dim=0)
        sample_count += 1

        if sample_count == num_sample:
            # Last batch
            y_zero_pad = torch.zeros((buffer_size, seq_len, HEIGHT, WIDTH), dtype=torch.float32)
            y_pred_buffer = torch.cat((y_pred_buffer, y_zero_pad), dim=0)

            for f in range(1, seq_len):
                # Last input sequence
                y_pred = y_pred_buffer[batch_i+b+f, frame_i].sum(0) / (seq_len-f)
                ensemble_i = torch.cat((ensemble_i, i[-1][f].reshape(1, 1, 2)), dim=0)
                ensemble_y_pred = torch.cat((ensemble_y_pred, y_pred.reshape(1, 1, HEIGHT, WIDTH)), dim=0)

    # Predict
    tmp_pred = predict(ensemble_i, y_pred=ensemble_y_pred, img_scaler=img_scaler)
    for key in tmp_pred.keys():
        tracknet_pred_dict[key].extend(tmp_pred[key])

    # Update buffer, keep last predictions for ensemble in next iteration
    y_pred_buffer = y_pred_buffer[-buffer_size:]


/root/TrackNetV3


  0%|                                                    | 0/21 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 21/21 [00:28<00:00,  1.36s/it]


In [8]:
%cd {HOME}/TrackNetV3
from utils.test import generate_inpaint_mask

inpaintnet.eval()
seq_len = inpaintnet_seq_len
tracknet_pred_dict['Inpaint_Mask'] = generate_inpaint_mask(tracknet_pred_dict, th_h=h*0.05)
inpaint_pred_dict = {'Frame':[], 'X':[], 'Y':[], 'Visibility':[]}

/root/TrackNetV3


In [9]:
# Create dataset with overlap sampling for temporal ensemble
dataset = Shuttlecock_Trajectory_Dataset(seq_len=seq_len, sliding_step=1, data_mode='coordinate', pred_dict=tracknet_pred_dict)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False)
weight = get_ensemble_weight(seq_len, eval_mode)

# Init buffer params
num_sample, sample_count = len(dataset), 0
buffer_size = seq_len - 1
batch_i = torch.arange(seq_len) # [0, 1, 2, 3, 4, 5, 6, 7]
frame_i = torch.arange(seq_len-1, -1, -1) # [7, 6, 5, 4, 3, 2, 1, 0]
coor_inpaint_buffer = torch.zeros((buffer_size, seq_len, 2), dtype=torch.float32)

for step, (i, coor_pred, inpaint_mask) in enumerate(tqdm(data_loader)):
    coor_pred, inpaint_mask = coor_pred.float(), inpaint_mask.float()
    b_size = i.shape[0]
    with torch.no_grad():
        coor_inpaint = inpaintnet(coor_pred.cuda(), inpaint_mask.cuda()).detach().cpu()
        coor_inpaint = coor_inpaint * inpaint_mask + coor_pred * (1-inpaint_mask)
    
    # Thresholding
    th_mask = ((coor_inpaint[:, :, 0] < COOR_TH) & (coor_inpaint[:, :, 1] < COOR_TH))
    coor_inpaint[th_mask] = 0.

    coor_inpaint_buffer = torch.cat((coor_inpaint_buffer, coor_inpaint), dim=0)
    ensemble_i = torch.empty((0, 1, 2), dtype=torch.float32)
    ensemble_coor_inpaint = torch.empty((0, 1, 2), dtype=torch.float32)
    
    for b in range(b_size):
        if sample_count < buffer_size:
            # Imcomplete buffer
            coor_inpaint = coor_inpaint_buffer[batch_i+b, frame_i].sum(0)
            coor_inpaint /= (sample_count+1)
        else:
            # General case
            coor_inpaint = (coor_inpaint_buffer[batch_i+b, frame_i] * weight[:, None]).sum(0)
        
        ensemble_i = torch.cat((ensemble_i, i[b][0].view(1, 1, 2)), dim=0)
        ensemble_coor_inpaint = torch.cat((ensemble_coor_inpaint, coor_inpaint.view(1, 1, 2)), dim=0)
        sample_count += 1

        if sample_count == num_sample:
            # Last input sequence
            coor_zero_pad = torch.zeros((buffer_size, seq_len, 2), dtype=torch.float32)
            coor_inpaint_buffer = torch.cat((coor_inpaint_buffer, coor_zero_pad), dim=0)
            
            for f in range(1, seq_len):
                coor_inpaint = coor_inpaint_buffer[batch_i+b+f, frame_i].sum(0)
                coor_inpaint /= (seq_len-f)
                ensemble_i = torch.cat((ensemble_i, i[-1][f].view(1, 1, 2)), dim=0)
                ensemble_coor_inpaint = torch.cat((ensemble_coor_inpaint, coor_inpaint.view(1, 1, 2)), dim=0)

    # Thresholding
    th_mask = ((ensemble_coor_inpaint[:, :, 0] < COOR_TH) & (ensemble_coor_inpaint[:, :, 1] < COOR_TH))
    ensemble_coor_inpaint[th_mask] = 0.

    # Predict
    tmp_pred = predict(ensemble_i, c_pred=ensemble_coor_inpaint, img_scaler=img_scaler)
    for key in tmp_pred.keys():
        inpaint_pred_dict[key].extend(tmp_pred[key])
    
    # Update buffer, keep last predictions for ensemble in next iteration
    coor_inpaint_buffer = coor_inpaint_buffer[-buffer_size:]

100%|███████████████████████████████████████████| 21/21 [00:01<00:00, 16.89it/s]


In [10]:
from utils.general import write_pred_csv
# Write csv file
pred_dict = inpaint_pred_dict if inpaintnet is not None else tracknet_pred_dict
write_pred_csv(pred_dict, save_file=out_csv_file)

In [11]:
import cv2

import numpy as np

from typing import Optional, Tuple
from dataclasses import dataclass

@dataclass(frozen=True)
class Point:
    x: float
    y: float

    @property
    def int_xy_tuple(self) -> Tuple[int, int]:
        return int(self.x), int(self.y)

@dataclass(frozen=True)
class Rect:
    x: float
    y: float
    width: float
    height: float

    @property
    def top_left(self) -> Point:
        return Point(x=self.x, y=self.y)

    @property
    def bottom_right(self) -> Point:
        return Point(x=self.x + self.width, y=self.y + self.height)

    @property
    def bottom_center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y + self.height)
    
    
@dataclass(frozen=True)
class Color:
    r: int
    g: int
    b: int

    @property
    def bgr_tuple(self) -> Tuple[int, int, int]:
        return self.b, self.g, self.r

def draw_rect(image: np.ndarray, rect: Rect, color: Color, thickness: int = 2) -> np.ndarray:
    cv2.rectangle(image, rect.top_left.int_xy_tuple, rect.bottom_right.int_xy_tuple, color.bgr_tuple, thickness)
    return image

def draw_rects(frame, rects, color=Color(r=0,g=255,b=0)):
    annotated_frame = frame
    for rect in rects:
        annotated_frame = draw_rect(image=annotated_frame,
                                    rect=rect,
                                    color=color,
                                    thickness=2)
    return annotated_frame

def group_and_average(arr, threshold):
    groups = []  # Store grouped numbers

    current_group = [arr[0]]
    for i in range(1, len(arr)):
        if arr[i] - arr[i - 1] <= threshold:
            current_group.append(arr[i])
        else:
            groups.append(current_group)  # Save the completed group
            current_group = [arr[i]]  # Start a new group

    groups.append(current_group)  # Add the last group

    # Calculate the average for each group
    averages = [int(np.mean(group)) for group in groups]
    return averages

## Get shots from csv

In [12]:
%cd {HOME}
import supervision as sv
import csv

frame_to_xy = {}
bounding_box_annotator = sv.BoundingBoxAnnotator()

with open('./TrackNetV3/prediction/rally_ball.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        frame_to_xy[int(row["Frame"])] = [int(row["X"]), int(row["Y"])]


frame_indices = np.arange(len(frame_to_xy))
y = [f[1] for f in frame_to_xy.values()]
y_1 = np.gradient(y)
y_2 = np.gradient(y_1)
mid_shot_frames = group_and_average([idx for idx in frame_indices if y_2[idx] < -3],threshold=5)
shot_frames = []
for frame in mid_shot_frames:
    for idx in range(frame-10,frame+3):
        shot_frames.append(idx)
        

/root


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


## Splitting video into shots

In [13]:
%cd {HOME}
import supervision as sv

video_info = sv.VideoInfo.from_video_path(video_path="rally.mp4")

for idx,shot_idx in enumerate(mid_shot_frames):
    start_frame = max(shot_idx-10,0)
    end_frame = min(shot_idx+10,video_info.total_frames - 1)
    total_frames = end_frame - start_frame + 1
    shot_video_info = sv.VideoInfo(width=video_info.width,height=video_info.height,fps=30,total_frames=total_frames)
    with sv.VideoSink(target_path=f'output/shot{idx}.mp4',video_info=shot_video_info) as sink:
        for frame in sv.get_video_frames_generator(source_path="rally.mp4",start=start_frame,end=end_frame):
            sink.write_frame(frame)
    print(f'done shot {idx}',end='\r')

/root


## Tracking players

In [21]:
%cd {HOME}
import supervision as sv
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
# results = model()

/root


## Annotating clip

In [22]:
import supervision as sv
def callback(frame, idx):
    x,y = frame_to_xy[idx]
    color = Color(r=255,g=0,b=0) if idx in shot_frames else Color(r=0,g=255,b=0)
    annotated_frame = draw_rects(frame, [Rect(x-10,y-10,20,20)],color=color)
    results = model(frame)
    return annotated_frame


sv.process_video(source_path='./rally.mp4', target_path='./output/rally_annotated.mp4', callback=callback)


0: 384x640 2 persons, 34.3ms
Speed: 3.4ms preprocess, 34.3ms inference, 136.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6.2ms
Speed: 1.2ms preprocess, 6.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.1ms
Speed: 1.2ms preprocess, 6.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

## Plotting Shuttle Trajectory

In [16]:
%cd {HOME}
import matplotlib.pyplot as plt
import numpy as np
frame_to_xy = {}

with open('./TrackNetV3/prediction/rally_ball.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        frame_to_xy[int(row["Frame"])] = [int(row["X"]), int(row["Y"])]

frame = np.arange(len(frame_to_xy))
x = [f[0] for f in frame_to_xy.values()]
y = [f[1] for f in frame_to_xy.values()]
y_1 = np.gradient(y)
y_2 = np.gradient(y_1)
shot_frames = [idx for idx in frame if y_2[idx] < -3]
# print()
# # print(np.where(np.diff(np.sign(y_2)) != 0))
# y_3 = np.gradient(y_2)


# # plt.plot(frame, x,color="red")
# # plt.plot(frame, y,color="green")
# # plt.plot(frame, y_1,color="blue")
# plt.plot(frame, y_2,color="purple")
# # plt.plot(frame, y_3,color="red")
# # plt.show()

# 30 f / s, frame 


/root
